In [11]:
# imports
import pandas as pd
import os # use this to access your environment variables
import requests # this will be used to call the APIs
import json
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')
df=pd.read_csv('Paris_data.csv')
df=df.head(500) # limiting my data to 500 to reduce the API request and saving it as a new data frame for this notebook 
df

,name,empty_slots,free_bikes,latitude,longitude,timestamp
0,Benjamin Godard - Victor Hugo,34,1,48.865983,2.275725,2023-12-06T02:45:12.426000Z
1,André Mazet - Saint-André des Arts,29,26,48.853756,2.339096,2023-12-06T02:45:12.322000Z
2,Faubourg Du Temple - Republique,31,5,48.867872,2.364898,2023-12-06T02:45:09.480000Z
3,Beaux-Arts - Bonaparte,19,1,48.856452,2.334852,2023-12-06T02:45:09.991000Z
4,Toudouze - Clauzel,18,3,48.879296,2.337360,2023-12-06T02:45:10.107000Z
...,...,...,...,...,...,...
495,Mozart - Jasmin,54,6,48.852861,2.268484,2023-12-06T02:45:10.028000Z
496,Boulets - Faubourg Saint-Antoine,20,22,48.849259,2.391755,2023-12-06T02:45:11.226000Z
497,Haies - Réunion,15,7,48.853869,2.402426,2023-12-06T02:45:10.874000Z
498,Maine - Antoine Bourdelle,16,1,48.843085,2.320264,2023-12-06T02:45:12.399000Z


In [12]:
def get_venues_fs(latitude, longitude):
    """
    Get venues from foursquare with coordinates.
    Args:
        latitude (float): latitude for query (must be combined with longitude)
        longitude (float): longitude for query (must be combined with latitude)
            
    Returns:
        response: response object from the requests library.
    """
    URL = 'https://api.foursquare.com/v3/places/search'
    # Prepare parameters for the API request
    params = {
        'll': f'{latitude},{longitude}',  # latitude and longitude
        'radius': 1000,  # radius around the location
        'categories':["13000,10000,12013,15008"]
            }
    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
                }

    response = requests.get(URL,params=params,headers=headers)
    return response

In [13]:
four_sq_data=[]

counter=0 # to count each iteration

max_count=20 # the max count before saving data to a file in the libarary

id=0 # for name of the file based on iteration

for index, row in df.iterrows():
    # Extract latitude and longitude from the current row
    lat, lon = row['latitude'], row['longitude']
    res = get_venues_fs(lat,lon)
    if res.status_code == 200:
        data = res.json()
        four_sq_data.append(data)# add data from each coordinate to a list
        if (counter%max_count==0) and (counter>0): # condition for interval of monitoring
            print(str(counter)+ " rows has been processed") # statement to see the progress of the data being pulled
            # saving the raw json file
            with open(f'four_sq_data_{id}.json', 'w') as f:
                json.dump(four_sq_data, f)
                id+=1 # incrementing the file number
    else:
        print(f"Error: {res.status_code}, {res.text}")
    # if index>=max_count: only for testing phase to stop the iterations
    #     break
    counter+=1
with open(f'four_sq_data_final.json', 'w') as f:
     json.dump(four_sq_data, f)


20 rows has been processed
40 rows has been processed
60 rows has been processed
80 rows has been processed
100 rows has been processed
120 rows has been processed
140 rows has been processed
160 rows has been processed
180 rows has been processed
200 rows has been processed
220 rows has been processed
240 rows has been processed
260 rows has been processed
280 rows has been processed
300 rows has been processed
320 rows has been processed
340 rows has been processed
360 rows has been processed
380 rows has been processed
400 rows has been processed
420 rows has been processed
440 rows has been processed
460 rows has been processed
480 rows has been processed


In [14]:
print(len(four_sq_data))

500


###### Below we are doing one final dump to get the remainder of data that does not match the conditional statement to get a json file with all 500 Data points

In [15]:
with open(f'four_sq_data_final.json', 'w') as f:
     json.dump(four_sq_data, f)

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating